## 评测几款读取音频的方式的速度差异

In [24]:
import wave
import numpy as np
from scipy.io import wavfile
import librosa

In [2]:
def read_wav_data_by_wave(filename):
    '''
    读取一个wav文件，返回声音信号的时域谱矩阵和播放时间
    '''
    wav = wave.open(filename,"rb") # 打开一个wav格式的声音文件流
    num_frame = wav.getnframes() # 获取帧数
    num_channel=wav.getnchannels() # 获取声道数
    framerate=wav.getframerate() # 获取帧速率
    num_sample_width=wav.getsampwidth() # 获取实例的比特宽度，即每一帧的字节数
    str_data = wav.readframes(num_frame) # 读取全部的帧
    wav.close() # 关闭流
    wave_data = np.fromstring(str_data, dtype = np.short) # 将声音文件数据转换为数组矩阵形式
    wave_data.shape = -1, num_channel # 按照声道数将数组整形，单声道时候是一列数组，双声道时候是两列的矩阵
    wave_data = wave_data.T # 将矩阵转置
    #wave_data = wave_data 
    return wave_data, framerate

In [48]:
def read_wav_data_by_scipy(filename):
    sampling_freq,audio = wavfile.read(filename)
    return audio,sampling_freq
    
def read_wav_data_by_librosa(filename):
    data,_ = librosa.core.load(filename,sr=16000)
    return data,_

In [50]:
read_wav_data_by_librosa(path[0]+".wav")

(array([-0.00213623, -0.00393677, -0.00549316, ..., -0.0007019 ,
        -0.00115967, -0.00079346], dtype=float32), 16000)

In [43]:
read_wav_data_by_wave(path[0]+".wav")

c:\users\sailist\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


(array([[ -70, -129, -180, ...,  -23,  -38,  -26]], dtype=int16), 16000)

In [47]:
read_wav_data_by_scipy(path[0]+".wav")

(array([ -70, -129, -180, ...,  -23,  -38,  -26], dtype=int16), 16000)

## 准备数据集

In [5]:
import os
import acoustic.Reader as Reader

In [6]:
z200 = Reader.Z200()

In [7]:
path = z200.load_from_path("../Dataset/z200")

In [11]:
path[:4]

['C:\\E\\jupyter_notebook\\voice_reco\\Dataset\\z200\\G0002\\session01\\T0055G0002S0001',
 'C:\\E\\jupyter_notebook\\voice_reco\\Dataset\\z200\\G0002\\session01\\T0055G0002S0002',
 'C:\\E\\jupyter_notebook\\voice_reco\\Dataset\\z200\\G0002\\session01\\T0055G0002S0004',
 'C:\\E\\jupyter_notebook\\voice_reco\\Dataset\\z200\\G0002\\session01\\T0055G0002S0005']

In [12]:
import time

## 测试wave

In [64]:
start = time.clock()
for i in path:
    read_wav_data_by_wave(i+".wav")
end = time.clock()
print(f"use {end-start} seconds.")

c:\users\sailist\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


use 0.5578425353205034 seconds.


## 测试scipy

In [65]:
start = time.clock()
for i in path:
    read_wav_data_by_scipy(i+".wav")
end = time.clock()
print(f"use {end-start} seconds.")

use 0.45256812690428205 seconds.


## 测试librosa

In [55]:
start = time.clock()
for i in path[:10]:
    read_wav_data_by_librosa(i+".wav")
end = time.clock()
print(f"use {end-start} seconds.")

use 0.6088633135516375 seconds.


可见由于libsora封装功能过于复杂，读取大量数据较为缓慢，而scipy应该较wave有些许优化，因此读取更加快速